## Import Dependencies and Setup Enviroment

In [1]:
import os
import openai
import json
import time
import pandas as pd
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.prompts import PromptTemplate

In [2]:
_ = load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

## Generate Seed Data via LLM

In [3]:
def generate_home_listings(prompt):
    system_prompt = """
    You are a real estate listing assistant specializing in crafting compelling and informative home descriptions. Your task is to generate real estate listings in **valid JSON format** based on user-provided details.
    
    Each listing should strictly follow this JSON format:

    {
        "Address": "[Address"]
        "Neighborhood": "[Neighborhood Name]",
        "Price": "$[Price]",
        "Bedrooms": [Number of Bedrooms],
        "Bathrooms": [Number of Bathrooms],
        "House_Size": "[Size in sqft]",
        "Description": "[Compelling property description]",
        "Neighborhood_Description": "[Engaging neighborhood description]"
    }

    **Rules:**
    - Return output in **pure JSON format** without any extra text.
    - Ensure values are properly formatted.
    - Use double quotes `" "` for JSON keys and values.

    Address: Generate a fake address with the Neighborhood as the city. Ex. 225 W 38th Street, {Neighborhood}
    
    Description:
    Provide a captivating yet concise property description, emphasizing key selling points such as architectural style, unique features, energy efficiency, or location benefits. Use persuasive language to make the home appealing to potential buyers. Highlight amenities, natural lighting, interior design elements, and lifestyle advantages.
    
    Neighborhood Description:
    Offer a brief, engaging description of the neighborhood. Highlight aspects such as community atmosphere, parks, restaurants, transportation, and local attractions. Focus on features that would be appealing to potential buyers considering the area.
    
    Maintain a professional and inviting tone, ensuring the listing feels aspirational yet grounded in reality. Avoid exaggeration while emphasizing the home’s best features.
    """
    
    try:
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": system_prompt
          },
          {
            "role": "user",
            "content": prompt
          }
          ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )

        raw_response = response.choices[0].message.content.strip()  

        if raw_response.startswith("{") and raw_response.endswith("}"):
            listing = json.loads(raw_response)  
        else:
            raise ValueError("OpenAI did not return valid JSON")

        return listing

    except Exception as e:
        return f"An error occurred: {e}" 

def generate_multiple_listings(prompts):
    listings = []

    for i, prompt in enumerate(prompts):
        print(f"Generating listing {i+1}/{len(prompts)}...")
        listing = generate_home_listings(prompt)
        
        if "An error occurred" not in listing:
            listings.append(listing)
        else:
            print(f"Error in listing {i+1}: {listing}")

        time.sleep(1)  

    with open("listings.json", "w") as f:
        json.dump(listings, f, indent=4)

    print(f"Saved {len(listings)} listings to listings.json")

    return listings

In [4]:
prompts = [
    "Generate a listing for a 4-bedroom, 2 bathroom house, with 4,000 sqft in Baltimore, priced at $400,000. The home has a parking pad and is near the park.",
    "Generate a listing for a 2-bedroom, 1 bathroom house, with 1,200 sqft in Washington DC, priced at $600,000. The home has a back patio and has street parking.",
    "Generate a listing for a 6-bedroom, 3 bathroom house, with 8,000 sqft in Dallas, priced at $500,000. The home has a large backyard and has three garages.",
    "Generate a listing for a 1-bedroom, 1 bathroom apartment, with 400 sqft in New York, priced at $800,000. The apartment has bathroom kitchen and features a window.",
    "Generate a listing for a 2-bedroom, 1 bathroom house, with 1,100 sqft in Baltimore, priced at $205,000. The home has a view of the harbor and is walking distance to the stadiums.",
    "Generate a listing for a 2-bedroom, 1 bathroom house, with 1,100 sqft in San Francisco, priced at $812,000. The home has is close to public transit and is near a park.",
    "Generate a listing for a 3-bedroom, 1 bathroom apartment, with 1,100 sqft in Chicago, priced at $700,000. The home is an apartment and has a view of the river.",
    "Generate a listing for a 3-bedroom, 2 bathroom house, with 2,100 sqft in New York, priced at $500,000. The home is in upstate New York and is near a wings restaurant.",
    "Generate a listing for a 3-bedroom, 2 bathroom house, with 1,100 sqft in Baltimore, priced at $190,000. The home near Canton square and has many dining options near it.",
    "Generate a listing for a 2-bedroom, 2 bathroom house, with 1,700 sqft in Dallas, priced at $195,000. The home is near the PGA headquarters and offers many opportunities to golf.",
    "Generate a listing for a 4-bedroom, 2 bathroom house, with 1,900 sqft in Eldersburg, priced at $450,000. The home is in a great school district.",
    "Generate a listing for a 4-bedroom, 2 bathroom house, with 1,700 sqft in Phoenix, priced at $400,000. The home is in the desert, has great views and is in a valley.",
    "Generate a listing for a 6-bedroom, 2 bathroom house, with 6,700 sqft in Dallas, priced at $600,000. The home is in the suberbs of Dallas and is only a 25 min drive from the city.",
    "Generate a listing for a 1-bedroom, 1 bathroom apartment, with 900 sqft in New York, priced at $800,000. The home is an apartment and is a 15 min walk from the park.",
    "Generate a listing for a 1-bedroom, 1 bathroom apartment, with 1,200 sqft in Miami, priced at $800,000. The home is an apartment and is a 15 min walk from the beach.",
]

listings = generate_multiple_listings(prompts)
print(json.dumps(listings[:2], indent=4))

Generating listing 1/15...
Generating listing 2/15...
Generating listing 3/15...
Generating listing 4/15...
Generating listing 5/15...
Generating listing 6/15...
Generating listing 7/15...
Generating listing 8/15...
Generating listing 9/15...
Generating listing 10/15...
Generating listing 11/15...
Generating listing 12/15...
Generating listing 13/15...
Generating listing 14/15...
Generating listing 15/15...
Saved 15 listings to home_listings.json
[
    {
        "Address": "123 Oak Street, Baltimore",
        "Neighborhood": "Baltimore",
        "Price": "$400,000",
        "Bedrooms": 4,
        "Bathrooms": 2,
        "House_Size": "4,000 sqft",
        "Description": "Welcome to your spacious 4-bedroom, 2-bathroom oasis in Baltimore! This charming home features a parking pad for convenience and easy access. With 4,000 sqft of living space, this property offers ample room for relaxation and entertainment. Enjoy the comfort of modern living in a serene neighborhood.",
        "Neighbo

## Storing Listings in Vector Database

In [13]:
with open("listings.json", "r") as f:
    listings = json.load(f)

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = []

for listing in listings:
    full_text = f"""
    Address: {listing["Address"]}
    Neighborhood: {listing["Neighborhood"]}
    Price: {listing["Price"]}
    Bedrooms: {listing["Bedrooms"]}
    Bathrooms: {listing["Bathrooms"]}
    House Size: {listing["House_Size"]}
    
    Description: {listing["Description"]}
    Neighborhood Description: {listing["Neighborhood_Description"]}
    """

    chunks = text_splitter.split_text(full_text)

    for chunk in chunks:
        doc = Document(page_content=chunk, metadata={
            "Address": listing["Address"],
            "Neighborhood": listing["Neighborhood"],
            "Price": listing["Price"],
            "Bedrooms": listing["Bedrooms"],
            "Bathrooms": listing["Bathrooms"],
            "House_Size": listing["House_Size"],
            "Description": listing["Description"],
            "Neighborhood_Description": listing["Neighborhood_Description"]
        })
        documents.append(doc)

print(f"documents: {documents}")

documents: [Document(page_content='Address: 123 S Main Street, Baltimore\n    Neighborhood: Baltimore\n    Price: $400,000\n    Bedrooms: 4\n    Bathrooms: 2\n    House Size: 4,000 sqft\n    \n    Description: Welcome to this spacious 4-bedroom, 2-bathroom home in Baltimore! This charming house boasts ample natural lighting, a parking pad for your convenience, and a prime location near the park. With 4,000 sqft of living space, this property offers the perfect blend of comfort and functionality.\n    Neighborhood Description: Nestled in the vibrant city of Baltimore, this neighborhood offers a perfect balance of urban convenience and suburban tranquility. Enjoy easy access to local parks, restaurants, and community amenities. Experience the charm of city living while being surrounded by green spaces and a welcoming community atmosphere.', metadata={'Address': '123 S Main Street, Baltimore', 'Neighborhood': 'Baltimore', 'Price': '$400,000', 'Bedrooms': 4, 'Bathrooms': 2, 'House_Size': '

In [14]:
persist_path = "./vector_db"
embedding_model = OpenAIEmbeddings()
db = Chroma.from_documents(documents, embedding_model, persist_directory=persist_path)
db.persist()

In [15]:
# Load the persisted Chroma vector database
embedding_model = OpenAIEmbeddings()
persist_path = "./vector_db"
db = Chroma(persist_directory=persist_path, embedding_function=embedding_model)

## Simulated Conversation

In [57]:
questions = [
    "Is there a specific Neighborhood you would like to live in?" ,
    "What is the maximum price you're willing to pay for the home?",
    "How many bedrooms would you like the house to have?",
    "How many bathrooms would you like the house to have?",
    "How big do you want your house to be?", 
    "What are the 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"  
]

answers = [
    "Baltimore",
    "I would like to pay between $200,000 and $400,000",
    "3 bedrooms.",
    "3 full bathrooms.",
    "A comfortable 3,000 square foot house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [58]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_bedrooms",
            "description": (
                "Always extract the exact number of bedrooms from user input as an **integer**. "
                "If the response contains words like 'three bedrooms' or '3 bedrooms', return **only** the number 3. "
                "Do not include text. **Never add this information to Features.** "
                "If a range is provided (e.g., '3 to 4 bedrooms'), extract only the **minimum** value."
            ),
            "parameters": {
                "type": "object",
                "properties": {"Bedrooms": {"type": "integer"}},
                "required": ["Bedrooms"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_bathrooms",
            "description": (
                "Always extract the exact number of bathrooms from user input as an **integer**. "
                "If the response contains words like 'three bathrooms' or '3 full bathrooms', return **only** the number 3. "
                "If half-bathrooms are mentioned, round them up to the nearest whole number. "
                "Ensure the value is always extracted, even if the response is already structured."
            ),
            "parameters": {
                "type": "object",
                "properties": {"Bathrooms": {"type": "integer"}},
                "required": ["Bathrooms"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_neighborhood",
            "description": (
                "Always extract the exact neighborhood or city name from the user's input as a structured string. "
                "Even if the user gives a clear city (e.g., 'Baltimore MD'), extract it explicitly. "
                "If the user provides a vague response (e.g., 'somewhere near the city'), return the closest reasonable neighborhood name."
            ),
            "parameters": {
                "type": "object",
                "properties": {"Neighborhood": {"type": "string"}},
                "required": ["Neighborhood"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_price",
            "description": (
                "Extract the maximum budget from the user input as an **integer**. "
                "If the response includes a price range (e.g., '$600,000 - $700,000'), extract only the **highest** value."
            ),
            "parameters": {
                "type": "object",
                "properties": {"Price": {"type": "integer"}},
                "required": ["Price"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_house_size",
            "description": "Extracts the house size in square feet from user input.",
            "parameters": {
                "type": "object",
                "properties": {"House_Size": {"type": "string"}},
                "required": ["House_Size"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_features",
            "description": "Extracts specific features desired in the home from user input.",
            "parameters": {
                "type": "object",
                "properties": {"Features": {"type": "array", "items": {"type": "string"}}},
                "required": ["Features"]
            }
        }
    }
]

def collect_structured_preferences():
    """Extract structured preferences from user responses and return in vector database query format."""
    
    messages = [
        {
            "role": "system",
            "content": (
                "You are an AI assistant extracting structured real estate search criteria. "
                "Your goal is to extract **exact numbers** for bedrooms and bathrooms, ensuring they are **integers**. "
                "Treat half bathrooms as full bathrooms by rounding up. "
                "Ensure that all extracted values match their intended categories."
            )
        }
    ]
    
    structured_criteria = {
        "Neighborhood": "",
        "Price": None,
        "Bedrooms": None,
        "Bathrooms": None,
        "House_Size": "",
        "Features": []
    }
    
    for question, answer, tool in zip(questions, answers, tools):
        formatted_qa = f"Q: {question}\nA: {answer}"
        print(f"formatted_qa: {formatted_qa}")
        messages.append({"role": "user", "content": formatted_qa})
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
            tools=[tool],
            tool_choice="auto"
        )
        
        # Extract tool call from response
        tool_calls = response["choices"][0].get("message", {}).get("tool_calls")
        
        if tool_calls:
            for tool_call in tool_calls:
                tool_name = tool_call["function"]["name"]
                tool_response = json.loads(tool_call["function"]["arguments"])

                # Ensure correct updates
                if tool_name == "extract_bedrooms":
                    structured_criteria["Bedrooms"] = tool_response.get("Bedrooms")
                elif tool_name == "extract_bathrooms":
                    structured_criteria["Bathrooms"] = tool_response.get("Bathrooms")
                elif tool_name == "extract_house_size":
                    structured_criteria["House_Size"] = tool_response.get("House_Size")
                elif tool_name == "extract_neighborhood":
                    structured_criteria["Neighborhood"] = tool_response.get("Neighborhood")
                elif tool_name == "extract_price":
                    structured_criteria["Price"] = tool_response.get("Price")
                elif tool_name == "extract_features":
                    structured_criteria["Features"] = [
                        feature for feature in tool_response.get("Features", []) 
                        if not any(x in feature.lower() for x in ["bedroom", "bathroom"])
                    ]
        else:
            print(f"Tool not invoked for question: {question}")

    # Ensure Features is always a list
    if not isinstance(structured_criteria["Features"], list):
        structured_criteria["Features"] = [structured_criteria["Features"]]

    # Ensure integer conversion where needed
    try:
        structured_criteria["Bedrooms"] = int(structured_criteria["Bedrooms"]) if structured_criteria["Bedrooms"] else None
    except ValueError:
        structured_criteria["Bedrooms"] = None

    try:
        structured_criteria["Bathrooms"] = int(structured_criteria["Bathrooms"]) if structured_criteria["Bathrooms"] else None
    except ValueError:
        structured_criteria["Bathrooms"] = None

    page_content = " ".join([f"{key}: {value}" for key, value in structured_criteria.items() if value])

    return {
        "page_content": page_content,
        "metadata": structured_criteria
    }

query_data = collect_structured_preferences()

# Check output
print(query_data)

formatted_qa: Q: Is there a specific Neighborhood you would like to live in?
A: Baltimore
Tool not invoked for question: Is there a specific Neighborhood you would like to live in?
formatted_qa: Q: What is the maximum price you're willing to pay for the home?
A: I would like to pay between $200,000 and $400,000
formatted_qa: Q: How many bedrooms would you like the house to have?
A: 3 bedrooms.
formatted_qa: Q: How many bathrooms would you like the house to have?
A: 3 full bathrooms.
formatted_qa: Q: How big do you want your house to be?
A: A comfortable 3,000 square foot house with a spacious kitchen and a cozy living room.
formatted_qa: Q: What are the 3 most important things for you in choosing this property?
A: A quiet neighborhood, good local schools, and convenient shopping options.
{'page_content': "Neighborhood: Baltimore Price: 400000 Bathrooms: 3 House_Size: 3,000 square foot Features: ['3,000 square foot house', 'spacious kitchen', 'cozy living room']", 'metadata': {'Neighbor

### Query Vector Database

In [59]:
def query_vector_db(search_criteria):
    """Retrieve relevant listings using vector database search."""
    print(f"search_criteria: {search_criteria}")
    metadata = search_criteria.get("metadata", {})
    query_text = ' '.join([
            f'Neighborhood: {metadata.get("Neighborhood", "")} Importance: High',
            f'Price: {metadata.get("Price", "")} Importance: High',
            f'Bedrooms: {metadata.get("Bedrooms", "")} Importance: Medium',
            f'Bathrooms: {metadata.get("Bathrooms", "")} Importance: Medium',
            f'Features: {", ".join(metadata.get("Features", []))} Importance: Medium'
        ])    
    print(f"query_text: {query_text}")
    query_embedding = embedding_model.embed_query(query_text)
    
    # Perform vector search in the database
    results = db.similarity_search_by_vector(query_embedding, k=5)

    listings = [{
        "Address": result.metadata.get("Address", "Unkown"),
        "Neighborhood": result.metadata.get("Neighborhood", "Unknown"),
        "Price": result.metadata.get("Price", "Unknown"),
        "Bedrooms": result.metadata.get("Bedrooms", "Unknown"),
        "Bathrooms": result.metadata.get("Bathrooms", "Unknown"),
        "House_Size": result.metadata.get("House_Size", "Unknown"),
        "Description": result.metadata.get("Description", "Unknown"),
        "Neighborhood_Description": result.metadata.get("Neighborhood_Description", "Unknown"),
        "Features": result.metadata.get("Features", [])
    } for result in results]
    
    return pd.DataFrame(listings)

vector_results = query_vector_db(query_data)
vector_results

search_criteria: {'page_content': "Neighborhood: Baltimore Price: 400000 Bathrooms: 3 House_Size: 3,000 square foot Features: ['3,000 square foot house', 'spacious kitchen', 'cozy living room']", 'metadata': {'Neighborhood': 'Baltimore', 'Price': 400000, 'Bedrooms': None, 'Bathrooms': 3, 'House_Size': '3,000 square foot', 'Features': ['3,000 square foot house', 'spacious kitchen', 'cozy living room']}}
query_text: Neighborhood: Baltimore Importance: High Price: 400000 Importance: High Bedrooms: None Importance: Medium Bathrooms: 3 Importance: Medium Features: 3,000 square foot house, spacious kitchen, cozy living room Importance: Medium


,Address,Neighborhood,Price,Bedrooms,Bathrooms,House_Size,Description,Neighborhood_Description,Features
0,"123 S Main Street, Baltimore",Baltimore,"$400,000",4,2,"4,000 sqft","Welcome to this spacious 4-bedroom, 2-bathroom...","Nestled in the vibrant city of Baltimore, this...",[]
1,"123 E Main Street, Baltimore",Baltimore,"$205,000",2,1,"1,100 sqft",Charming 2-bedroom home with a breathtaking ha...,"Nestled in the heart of Baltimore, this vibran...",[]
2,"123 E Elm Street, Baltimore",Baltimore,"$190,000",3,2,"1,100 sqft","Welcome to this charming 3-bedroom, 2-bathroom...","Nestled in the vibrant Baltimore neighborhood,...",[]
3,"123 Main Street, Washington DC",Washington DC,"$600,000",2,1,"1,200 sqft",Charming 2-bedroom home in the heart of Washin...,"Nestled in Washington DC, this vibrant neighbo...",[]
4,"123 Oak Lane, Eldersburg",Eldersburg,"$450,000",4,2,"1,900 sqft","Welcome to this charming 4-bedroom, 2-bathroom...","Eldersburg, known for its excellent schools an...",[]


## LLM Results Presentation

In [60]:
input_variables=["neighborhood", "price", "bedrooms", "bathrooms", "features",
                     "listing_address", "listing_price", "listing_bedrooms", 
                     "listing_bathrooms", "listing_features", "listing_description"]
template="""
You are a real estate assistant providing personalized property descriptions. 

The buyer is looking for a property in {neighborhood} with a budget of around {price}. 
They prefer {bedrooms} bedrooms and {bathrooms} bathrooms, and they are particularly 
interested in features like {features}.

Below is a property listing:

- Address: {listing_address}
- Price: {listing_price}
- Bedrooms: {listing_bedrooms}
- Bathrooms: {listing_bathrooms}
- Features: {listing_features}
- Description: {listing_description}

### Task:
1. Rewrite the property description so it highlights aspects that match the buyer’s preferences.
2. Keep all factual details intact.
3. Maintain a professional and engaging tone.

### Output:
"""

property_description_prompt = PromptTemplate(input_variables=input_variables, template=template)

In [61]:
def generate_personalized_description(llm, user_prefs, listing):
    # Format the prompt using the template
    prompt = property_description_prompt.format(
        neighborhood=user_prefs.get("Neighborhood", "N/A"),
        price=user_prefs.get("Price", "N/A"),
        bedrooms=user_prefs.get("Bedrooms", "N/A"),
        bathrooms=user_prefs.get("Bathrooms", "N/A"),
        features=", ".join(user_prefs.get("Features", [])),

        listing_address=listing.get("Address", "N/A"),
        listing_price=listing.get("Price", "N/A"),
        listing_bedrooms=listing.get("Bedrooms", "N/A"),
        listing_bathrooms=listing.get("Bathrooms", "N/A"),
        listing_features=", ".join(listing.get("Features", [])),
        listing_description=listing.get("Description", "N/A")
    )

    
    response = llm.invoke(prompt) 
    return response

In [62]:
user_prefs = {
    "Neighborhood": query_data['metadata'].get("Neighborhood", ""),
    "Price": query_data['metadata'].get("Price", ""),
    "Bedrooms": query_data['metadata'].get("Bedrooms", ""),
    "Bathrooms": query_data['metadata'].get("Bathrooms", ""),
    "House_Size": query_data['metadata'].get("House_Size", ""),
    "Features": query_data['metadata'].get("Features", [])
}
listing = vector_results.iloc[0].to_dict()

print(f"user_prefs: {user_prefs}")
print(f"listing: {listing}")

user_prefs: {'Neighborhood': 'Baltimore', 'Price': 400000, 'Bedrooms': None, 'Bathrooms': 3, 'House_Size': '3,000 square foot', 'Features': ['3,000 square foot house', 'spacious kitchen', 'cozy living room']}
listing: {'Address': '123 S Main Street, Baltimore', 'Neighborhood': 'Baltimore', 'Price': '$400,000', 'Bedrooms': 4, 'Bathrooms': 2, 'House_Size': '4,000 sqft', 'Description': 'Welcome to this spacious 4-bedroom, 2-bathroom home in Baltimore! This charming house boasts ample natural lighting, a parking pad for your convenience, and a prime location near the park. With 4,000 sqft of living space, this property offers the perfect blend of comfort and functionality.', 'Neighborhood_Description': 'Nestled in the vibrant city of Baltimore, this neighborhood offers a perfect balance of urban convenience and suburban tranquility. Enjoy easy access to local parks, restaurants, and community amenities. Experience the charm of city living while being surrounded by green spaces and a welcom

In [63]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
personalized_desc = generate_personalized_description(llm, user_prefs, listing)
print(personalized_desc.content)

Welcome to this beautiful property located at 123 S Main Street in Baltimore! This 4-bedroom, 2-bathroom home offers a cozy living room, a spacious kitchen, and 3 bathrooms for your convenience. With a total of 4,000 square feet of living space, this house provides ample room for you and your family to spread out and relax. Additionally, the property features a parking pad for easy parking and is situated near a park, perfect for outdoor activities. Priced at $400,000, this home offers the ideal combination of comfort and functionality. Don't miss out on the opportunity to make this house your new home!
